In [3]:
import pandas as pd
import joblib
from underthesea import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from numpy import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack
from sqlalchemy import create_engine

In [41]:
df = pd.read_csv("Dataset_articles_NoID.csv")
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...
313315,https://laodong.vn/tlv-canh-doi/ba-lao-mu-nuoi...,"Bà lão mù nuôi con, cháu bị ảnh hưởng chất độc...","Bà Dương Thị Tuyết ở thị trấn Khe Sanh, huyện ...",NaN,"Thứ tư, 14/08/2013 06:57 (GMT+7)",Lâm Hưng Thơ,Tấm Lòng Vàng,['Chất độc']
313316,https://laodong.vn/ho-tro-bao-ve-chu-quyen-bie...,Trao 100 triệu đồng cho ngư dân Phú Yên,"Ngày 25.7, Đại diện Chương trình “Tấm lưới ngh...",NaN,"Thứ sáu, 26/07/2013 09:03 (GMT+7)",Lưu Phong,Tin hoạt động,['Phú Yên']
313317,https://laodong.vn/giup-do-cac-canh-doi-kho-kh...,Trao 2 “Mái ấm công đoàn” tới CNVCLĐ khó khăn ...,"Ngày 17.7, lãnh đạo Quỹ TLV Lao Động và LĐLĐ t...",NaN,"Thứ năm, 18/07/2013 07:37 (GMT+7)",Bảo Duy,Tin hoạt động,"['Mái ấm Công đoàn', 'Bắc Giang']"
313318,https://laodong.vn/tlv-canh-doi/hay-giup-do-mo...,Hãy giúp đỡ một nhà báo cần thay thận,Nhà báo Trần Đình Chính (tức Trần Hoài Thu) - ...,NaN,"Thứ năm, 20/06/2013 07:28 (GMT+7)",Hoàng Mạnh - Phạm Tùng,Tấm Lòng Vàng,"['Nhà báo', 'Báo']"


In [42]:
#Bỏ cột URL
df = df.drop("URL", axis=1)
#Bỏ cột Tags
df = df.drop("Tags", axis=1)

In [43]:
#Loại bỏ hàng NULL/NA
df = df.dropna(axis=0, how='any')
#Loại bỏ dòng trùng
df = df.drop_duplicates(keep='last', subset=['Title'])
#Reset index
df = df.reset_index(drop=True)
df

,Title,Summary,Contents,Date,Author(s),Category
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản
...,...,...,...,...,...,...
283381,Quảng Trị: Trao 300 triệu đồng cho người dân b...,"Sáng nay (7.10) tại tỉnh Quảng Trị, đồng chí N...",Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...,"Thứ hai, 07/10/2013 12:02 (GMT+7)",Hưng Thơ,Tin hoạt động
283382,Trao 100 triệu đồng hỗ trợ CNVCLĐ tỉnh Nghệ An...,"Sáng nay (7.10), tại Nghệ An, Quỹ Tấm lòng vàn...",Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...,"Thứ hai, 07/10/2013 11:53 (GMT+7)",Bảo Duy,Tin hoạt động
283383,Quỹ Tấm lòng vàng Lao Động: Trao 4 triệu đồng ...,Đại diện Quỹ Tấm lòng Lao Động và LĐLĐ Phú Yên...,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...,"Chủ nhật, 06/10/2013 19:33 (GMT+7)",L.Phong,Tin hoạt động
283384,Thêm những tấm lòng thơm thảo hướng về đồng bà...,"Tính đến chiều 5.10, Quỹ Tấm lòng vàng (TLV) L...","Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6...","Thứ bảy, 05/10/2013 15:51 (GMT+7)",Bảo Duy,Tin hoạt động


In [45]:
#Lọc bớt dữ liệu

In [46]:
#Đếm số lượng Category
num_unique_categories = df['Category'].nunique()

print(f'Số lượng giá trị duy nhất trong cột Category: {num_unique_categories}')

# Lấy các giá trị duy nhất trong cột Category
unique_categories = df['Category'].unique()

# Hiển thị kết quả
print(f'Các giá trị duy nhất trong cột Category: {unique_categories}')

categories = ['Bất động sản', 'Xã hội', 'Kinh doanh', 'Thời sự', 'Pháp luật', 'Giáo dục', 'Sức khỏe', 'Thể thao', 'Công nghệ', 'Du lịch']
result = df[df['Category'].isin(categories)]

# Nhóm các hàng theo giá trị của cột Category và lấy 1111 dòng đầu tiên cho mỗi nhóm
result = result.groupby('Category').head(1111)
result = result.reset_index(drop=True)
# Hiển thị kết quả
result


Số lượng giá trị duy nhất trong cột Category: 39
Các giá trị duy nhất trong cột Category: ['Bất động sản' 'Xã hội' 'Lao Động cuối tuần' 'Kinh doanh'
 'Sự kiện Bình luận' 'Lao Động & Đời sống' 'Thời sự' 'Thế giới' 'Lưu trữ'
 'Văn hóa - Giải trí' 'Video' 'Diễn đàn' 'Phóng sự' 'Tin bài nổi bật'
 'Pháp luật' 'Giáo dục' 'Thông tin tiện ích' 'Media' 'Tin địa phương'
 'Du lịch' 'Thông tin doanh nghiệp' 'Người Việt tử tế' 'Công đoàn'
 'Sổ tay kinh tế' 'Xe +' 'Thể thao' 'Lao Động Xuân' 'Tin bài xem thêm'
 'Sức khỏe' 'Tin bài liên quan' 'Gia đình - Hôn nhân'
 'Tản mạn - Chuyện dọc đường' 'Công nghệ' 'Tin tức việc làm' 'Bạn đọc'
 'Phóng sự - Điều tra' 'Tấm Lòng Vàng' 'Tin hoạt động' 'Quỹ TLV']


,Title,Summary,Contents,Date,Author(s),Category
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản
...,...,...,...,...,...,...
10005,Đề xuất thay đổi Champions League: “Cái tát và...,UEFA đang bị cho là lấy ý tưởng của Super Leag...,Nếu như việc các câu lạc bộ lớn quyết định thà...,"Thứ sáu, 15/04/2022 17:21 (GMT+7)",TAM NGUYÊN,Thể thao
10006,Tìm hiểu luật chơi golf: Di chuyển vật thể có ...,Người chơi golf không được thay đổi các điều k...,11.3. Cố tình di chuyển vật thể hoặc thay đổi ...,"Thứ sáu, 15/04/2022 17:12 (GMT+7)",TAM NGUYÊN,Thể thao
10007,Jeonbuk Hyundai Motors - Sydney FC: Đối đầu đỉ...,Jeonbuk Hyundai Motors vs Sydney FC hứa hẹn tạ...,Trận đấu giữa Jeonbuk Hyundai Motors vs Sydney...,"Thứ sáu, 15/04/2022 17:00 (GMT+7)",NGUYỄN ĐĂNG,Thể thao
10008,5 lý do khiến Barcelona thất bại trước Frankfurt,Barcelona đang đứng trước một mùa giải trắng t...,Barcelona chính thức đứt chuỗi 15 trận bất bại...,"Thứ sáu, 15/04/2022 15:39 (GMT+7)",Nguyễn Văn An,Thể thao


In [48]:
unique_categories = result['Category'].unique()
print(f'Các giá trị duy nhất trong cột Category: {unique_categories}')

Các giá trị duy nhất trong cột Category: ['Bất động sản' 'Xã hội' 'Kinh doanh' 'Thời sự' 'Pháp luật' 'Giáo dục'
 'Du lịch' 'Thể thao' 'Sức khỏe' 'Công nghệ']


In [49]:
#Viết hoa cột Author(s)
result.loc[:, "Author(s)"] = result["Author(s)"].str.upper()


In [53]:
df = result.copy()

In [54]:
#Chuyển từ tiếng việt sang anh
def convert_vietnamese_date(date_string):
    days = {
        'Thứ hai': 'Monday',
        'Thứ ba': 'Tuesday',
        'Thứ tư': 'Wednesday',
        'Thứ năm': 'Thursday',
        'Thứ sáu': 'Friday',
        'Thứ bảy': 'Saturday',
        'Chủ nhật': 'Sunday'
    }
    for vietnamese_day, english_day in days.items():
        date_string = date_string.replace(vietnamese_day, english_day)
    return date_string

df['Date'] = df['Date'].apply(convert_vietnamese_date)

In [55]:
def remove_timezone(date_string):
    return date_string.rsplit(' ', 1)[0]

df['Date'] = df['Date'].apply(remove_timezone)
df['Date'] = pd.to_datetime(df['Date'], format='%A, %d/%m/%Y %H:%M')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M')

In [58]:
df.insert(0, 'id', range(1, 1 + len(df)))
df

,id,Title,Summary,Contents,Date,Author(s),Category
0,1,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...",2022-05-20 08:56,PHƯƠNG NHIÊN,Bất động sản
1,2,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...",2022-05-20 08:10,GIA MIÊU,Bất động sản
2,3,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,2022-05-20 07:47,NGUYỄN TRƯỜNG,Bất động sản
3,4,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,2022-05-20 06:44,KIM NHUNG (T/H),Bất động sản
4,5,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,2022-05-19 15:30,HUYỀN NGUYỄN,Bất động sản
...,...,...,...,...,...,...,...
10005,10006,Đề xuất thay đổi Champions League: “Cái tát và...,UEFA đang bị cho là lấy ý tưởng của Super Leag...,Nếu như việc các câu lạc bộ lớn quyết định thà...,2022-04-15 17:21,TAM NGUYÊN,Thể thao
10006,10007,Tìm hiểu luật chơi golf: Di chuyển vật thể có ...,Người chơi golf không được thay đổi các điều k...,11.3. Cố tình di chuyển vật thể hoặc thay đổi ...,2022-04-15 17:12,TAM NGUYÊN,Thể thao
10007,10008,Jeonbuk Hyundai Motors - Sydney FC: Đối đầu đỉ...,Jeonbuk Hyundai Motors vs Sydney FC hứa hẹn tạ...,Trận đấu giữa Jeonbuk Hyundai Motors vs Sydney...,2022-04-15 17:00,NGUYỄN ĐĂNG,Thể thao
10008,10009,5 lý do khiến Barcelona thất bại trước Frankfurt,Barcelona đang đứng trước một mùa giải trắng t...,Barcelona chính thức đứt chuỗi 15 trận bất bại...,2022-04-15 15:39,NGUYỄN VĂN AN,Thể thao


In [60]:
# Xuất DataFrame ra tệp CSV
df.to_csv('NewData.csv', index=False)

In [66]:
df = result.copy()

In [67]:
# Tạo một từ điển ánh xạ giá trị cũ sang giá trị mới
mapping = {'Bất động sản': 1, 'Xã hội': 2, 'Kinh doanh': 3, 'Thời sự': 4, 'Pháp luật': 5, 'Giáo dục': 6, 'Sức khỏe': 7, 'Thể thao': 8, 'Công nghệ': 9, 'Du lịch': 10}
# Thay đổi giá trị của cột Category
df['Category'] = df['Category'].replace(mapping)

In [69]:

# Đổi tên các cột của DataFrame để khớp với tên của các cột trong bảng Posts
df = df.rename(columns={'Title': 'title', 'Summary': 'excerpt', 'Contents': 'content', 'Date': 'dateCreate', 'Author(s)': 'author', 'Category': 'category_id'})

In [71]:
df = df.assign(isEnabled=1, idUser_id=1)
df

,id,title,excerpt,content,dateCreate,author,category_id,isEnabled,idUser_id
0,1,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,1,1,1
1,2,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,1,1,1
2,3,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,1,1,1
3,4,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),1,1,1
4,5,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,1,1,1
...,...,...,...,...,...,...,...,...,...
10005,10006,Đề xuất thay đổi Champions League: “Cái tát và...,UEFA đang bị cho là lấy ý tưởng của Super Leag...,Nếu như việc các câu lạc bộ lớn quyết định thà...,"Thứ sáu, 15/04/2022 17:21 (GMT+7)",TAM NGUYÊN,8,1,1
10006,10007,Tìm hiểu luật chơi golf: Di chuyển vật thể có ...,Người chơi golf không được thay đổi các điều k...,11.3. Cố tình di chuyển vật thể hoặc thay đổi ...,"Thứ sáu, 15/04/2022 17:12 (GMT+7)",TAM NGUYÊN,8,1,1
10007,10008,Jeonbuk Hyundai Motors - Sydney FC: Đối đầu đỉ...,Jeonbuk Hyundai Motors vs Sydney FC hứa hẹn tạ...,Trận đấu giữa Jeonbuk Hyundai Motors vs Sydney...,"Thứ sáu, 15/04/2022 17:00 (GMT+7)",NGUYỄN ĐĂNG,8,1,1
10008,10009,5 lý do khiến Barcelona thất bại trước Frankfurt,Barcelona đang đứng trước một mùa giải trắng t...,Barcelona chính thức đứt chuỗi 15 trận bất bại...,"Thứ sáu, 15/04/2022 15:39 (GMT+7)",NGUYỄN VĂN AN,8,1,1


In [79]:
# Tạo kết nối đến cơ sở dữ liệu MySQL
engine = create_engine('mysql+mysqlconnector://root:@localhost/TVNews')

In [81]:
# Chèn dataframe vào bảng Posts trong cơ sở dữ liệu MySQL
start = 0
end = 100
while start < len(df):
    chunk = df.iloc[start:end]
    # insert chunk into mysql database
    chunk.to_sql('api_posts', engine, if_exists='append', index=False)
    start += 100
    end += 100

In [ ]:
#Train

In [101]:
df = result.copy()

In [102]:
#Bỏ cột id
df = df.drop("id", axis=1)
#Bỏ cột Contents
df = df.drop("Contents", axis=1)
#Bỏ cột Date
df = df.drop("Date", axis=1)
df

,Title,Summary,Author(s),Category
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...",PHƯƠNG NHIÊN,Bất động sản
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,GIA MIÊU,Bất động sản
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",NGUYỄN TRƯỜNG,Bất động sản
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,KIM NHUNG (T/H),Bất động sản
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,HUYỀN NGUYỄN,Bất động sản
...,...,...,...,...
10005,Đề xuất thay đổi Champions League: “Cái tát và...,UEFA đang bị cho là lấy ý tưởng của Super Leag...,TAM NGUYÊN,Thể thao
10006,Tìm hiểu luật chơi golf: Di chuyển vật thể có ...,Người chơi golf không được thay đổi các điều k...,TAM NGUYÊN,Thể thao
10007,Jeonbuk Hyundai Motors - Sydney FC: Đối đầu đỉ...,Jeonbuk Hyundai Motors vs Sydney FC hứa hẹn tạ...,NGUYỄN ĐĂNG,Thể thao
10008,5 lý do khiến Barcelona thất bại trước Frankfurt,Barcelona đang đứng trước một mùa giải trắng t...,NGUYỄN VĂN AN,Thể thao


In [103]:
# Tạo một danh sách stopword đơn giản cho tiếng Việt
stop_words = set(["là", "của", "và", "có", "được", "trong", "với", "cho", "đến"])
# Lấy một danh sách stopword cho tiếng Việt từ file
with open("vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    new_stop_words = set(f.read().splitlines())
    stop_words.update(new_stop_words)

# Định nghĩa hàm chuẩn hóa dữ liệu văn bản
def normalize_text(text):
    # Loại bỏ ký tự đặc biệt và chuyển đổi sang chữ thường
    text = "".join(char for char in text if char.isalnum() or char.isspace()).lower()
    # Tách từ
    words = word_tokenize(text)
    # Loại bỏ stopword
    words = [word for word in words if word not in stop_words]
    # Nối lại các từ đã xử lý
    return " ".join(words)

# Chuẩn hóa dữ liệu văn bản cho cột "Title" và "Summary"
df.loc[:, "Title"] = df["Title"].apply(normalize_text)
df.loc[:, "Summary"] = df["Summary"].apply(normalize_text)


In [105]:
df_copy = df.copy()
df

,Title,Summary,Author(s),Category
0,thông tin ngọc trinh mua đất bảo lộc chiêu trò...,lâm đồng lãnh đạo thành phố bảo lộc lâm đồng t...,PHƯƠNG NHIÊN,Bất động sản
1,lỗ hổng thẩm tra năng lực tài chính tham gia đ...,tphcm cưỡng chế thuế hai doanh nghiệp trúng đấ...,GIA MIÊU,Bất động sản
2,hoàn thiện dự án nhà ở xã hội cnlđ mua,hiện địa bàn tỉnh ninh bình 32 khu cụm công ng...,NGUYỄN TRƯỜNG,Bất động sản
3,chi tiết hồ sơ hoàn công 2022,hoàn công ý nghĩa đổi sổ đỏ thể hiện hiện trạn...,KIM NHUNG (T/H),Bất động sản
4,khởi không gian sống đẳng cấp đón sóng đầu tư ...,dự án nội đô an quý villa giải khát lý tưởng n...,HUYỀN NGUYỄN,Bất động sản
...,...,...,...,...
10005,đề xuất champions league tát má trái uefa,uefa ý tưởng super league áp dụng champions le...,TAM NGUYÊN,Thể thao
10006,luật chơi golf di chuyển vật thể ảnh hưởng bóng,golf ảnh hưởng đường đi bóng,TAM NGUYÊN,Thể thao
10007,jeonbuk hyundai motors sydney fc đối đầu đỉnh cao,jeonbuk hyundai motors vs sydney fc hứa hẹn tr...,NGUYỄN ĐĂNG,Thể thao
10008,5 barcelona thất bại frankfurt,barcelona đứng mùa giải trắng tay thua frankfu...,NGUYỄN VĂN AN,Thể thao


In [106]:
# Khởi tạo vectorizer
vectorizer = TfidfVectorizer()

# Tính toán TF-IDF cho tất cả các thuộc tính
vectorizer.fit(df['Title'] + df['Summary'])
title_tfidf = vectorizer.transform(df['Title'])
summary_tfidf = vectorizer.transform(df['Summary'])

# Kết hợp các ma trận TF-IDF lại với nhau
# (có thể sử dụng các phương pháp khác nhau để kết hợp)
features = hstack([title_tfidf, summary_tfidf])

In [107]:
# Khởi tạo encoder
encoder_category = OneHotEncoder(handle_unknown='ignore')
encoder_author = OneHotEncoder(handle_unknown='ignore')


# Mã hóa các thuộc tính phân loại
category_encoded = encoder_category.fit_transform(df[['Category']])
author_encoded = encoder_author.fit_transform(df[['Author(s)']])

# Kết hợp các ma trận lại với nhau
features = hstack([features, category_encoded])
features = hstack([features, author_encoded])

In [108]:
#Lưu file
joblib.dump(features, 'features.joblib')
features

<10010x33912 sparse matrix of type '<class 'numpy.float64'>'
	with 355727 stored elements in Compressed Sparse Row format>

In [109]:
features_new = joblib.load('features.joblib')
features_new

<10010x33912 sparse matrix of type '<class 'numpy.float64'>'
	with 355727 stored elements in Compressed Sparse Row format>

In [110]:
#Features là ma trận đặc trưng đã tính toán ở bước trước

# Khởi tạo mô hình
model = NearestNeighbors()

# Huấn luyện mô hình
model.fit(features)

# Dự đoán mức độ tương tự giữa các bài viết
# (ví dụ: tìm 10 bài viết tương tự nhất với bài viết đầu tiên)
distances, indices = model.kneighbors(features, n_neighbors=10)

In [112]:
print("Indices:", indices)
print("Distances:", distances)

Indices: [[    0   850   905 ...   371   435   647]
 [    1   168    15 ...    11   669   608]
 [    2   231    22 ...   452   111   518]
 ...
 [10007  9811  9690 ...  9895  9914  9892]
 [10008  8944  9849 ...  9163  9968  9915]
 [10009  9706  9476 ...  9891  9573  9866]]
Distances: [[0.         2.18978151 2.19560631 ... 2.24071816 2.24195205 2.24727363]
 [0.         1.52477213 1.52907909 ... 1.65776413 1.66086184 1.68270612]
 [0.         1.93869929 2.26693577 ... 2.30336053 2.30413922 2.30999498]
 ...
 [0.         1.61728007 1.6404974  ... 1.7160157  1.72499178 1.72606848]
 [0.         1.65622376 1.6715852  ... 1.76567463 1.78597367 1.80044473]
 [0.         1.69784875 1.72453458 ... 1.79335252 1.79362071 1.79893094]]


In [113]:
# Lưu mô hình vào file
joblib.dump(model, 'model.pkl')


['model.pkl']

In [124]:
split_indices = np.split(indices, indices.shape[1], axis=1)

# Tạo DataFrame từ các mảng con
data = {f'index_{i}': split_indices[i].ravel() for i in range(len(split_indices))}
df = pd.DataFrame(data)
df


,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9
0,0,850,905,409,698,995,619,371,435,647
1,1,168,15,468,564,78,553,11,669,608
2,2,231,22,327,527,620,17,452,111,518
3,3,338,1111,174,457,544,633,1068,306,102
4,4,789,1096,876,1011,9,979,505,989,986
...,...,...,...,...,...,...,...,...,...,...
10005,10005,9604,9996,9329,9697,9641,9997,9406,9541,9506
10006,10006,9663,9898,9902,9783,9073,9035,9072,8996,9696
10007,10007,9811,9690,9860,9727,9815,9896,9895,9914,9892
10008,10008,8944,9849,9492,8993,9716,10001,9163,9968,9915


In [125]:
df = df.add(1)
df

,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9
0,1,851,906,410,699,996,620,372,436,648
1,2,169,16,469,565,79,554,12,670,609
2,3,232,23,328,528,621,18,453,112,519
3,4,339,1112,175,458,545,634,1069,307,103
4,5,790,1097,877,1012,10,980,506,990,987
...,...,...,...,...,...,...,...,...,...,...
10005,10006,9605,9997,9330,9698,9642,9998,9407,9542,9507
10006,10007,9664,9899,9903,9784,9074,9036,9073,8997,9697
10007,10008,9812,9691,9861,9728,9816,9897,9896,9915,9893
10008,10009,8945,9850,9493,8994,9717,10002,9164,9969,9916


In [126]:
#Bỏ cột index_0
df = df.drop("index_0", axis=1)


In [127]:
df.insert(0, 'index_0', range(1, 1 + len(df)))

In [128]:
df = df.rename(columns={'index_0': 'idPost_id', 'index_1': 'id1', 'index_2': 'id2', 'index_3': 'id3', 'index_4': 'id4', 'index_5': 'id5', 'index_6': 'id6', 'index_7': 'id7', 'index_8': 'id8', 'index_9': 'id9'})
df

,idPost_id,id1,id2,id3,id4,id5,id6,id7,id8,id9
0,1,851,906,410,699,996,620,372,436,648
1,2,169,16,469,565,79,554,12,670,609
2,3,232,23,328,528,621,18,453,112,519
3,4,339,1112,175,458,545,634,1069,307,103
4,5,790,1097,877,1012,10,980,506,990,987
...,...,...,...,...,...,...,...,...,...,...
10005,10006,9605,9997,9330,9698,9642,9998,9407,9542,9507
10006,10007,9664,9899,9903,9784,9074,9036,9073,8997,9697
10007,10008,9812,9691,9861,9728,9816,9897,9896,9915,9893
10008,10009,8945,9850,9493,8994,9717,10002,9164,9969,9916


In [130]:
# Xuất DataFrame ra tệp CSV
df.to_csv('KetQuaDeXuat.csv', index=False, header=False)

In [131]:
duplicates = df['idPost_id'].duplicated().sum()
duplicates

0

In [132]:
# Chèn dataframe vào bảng Posts trong cơ sở dữ liệu MySQL
start = 0
end = 100
while start < len(df):
    chunk = df.iloc[start:end]
    # insert chunk into mysql database
    chunk.to_sql('api_relateds', engine, if_exists='append', index=False)
    start += 100
    end += 100

In [4]:
test = pd.read_csv("NewData.csv")



In [5]:
new_test = test.iloc[1]
new_test = pd.DataFrame(new_test).transpose()
new_test

,id,Title,Summary,Contents,Date,Author(s),Category
1,2,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...",2022-05-20 08:10,GIA MIÊU,Bất động sản


In [6]:
new_test

,id,Title,Summary,Contents,Date,Author(s),Category
1,2,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...",2022-05-20 08:10,GIA MIÊU,Bất động sản


In [7]:
# Chuẩn hóa dữ liệu văn bản cho cột "Title" và "Summary"
new_test.loc[:, "Title"] = new_test["Title"].apply(normalize_text)
new_test.loc[:, "Summary"] = new_test["Summary"].apply(normalize_text)


NameError: name 'normalize_text' is not defined

In [1]:
new_test

NameError: name 'new_test' is not defined

In [125]:
new_title_tfidf = vectorizer.transform(new_test['Title'])
new_summary_tfidf = vectorizer.transform(new_test['Summary'])
new_features = hstack([new_title_tfidf, new_summary_tfidf])


In [126]:
features

<10010x35122 sparse matrix of type '<class 'numpy.float64'>'
	with 371226 stored elements in Compressed Sparse Row format>

In [127]:
new_features

<1x33720 sparse matrix of type '<class 'numpy.float64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [128]:
new_category_encoded = encoder_category.transform(new_test[['Category']])
new_author_encoded = encoder_author.transform(new_test[['Author(s)']])
new_features = hstack([new_features, new_category_encoded])
new_features = hstack([new_features, new_author_encoded])

In [129]:
new_features

<1x35122 sparse matrix of type '<class 'numpy.float64'>'
	with 40 stored elements in Compressed Sparse Row format>

In [130]:
# Dự đoán mức độ tương tự giữa các bài viết


# Đọc mô hình từ file
readModel = joblib.load('model.pkl')

# (ví dụ: tìm 10 bài viết tương tự nhất với bài viết đầu tiên)
distances, indices = readModel.kneighbors(new_features[0], n_neighbors=10)

In [131]:
print(distances)

[[0.         1.51254516 1.52074305 1.58903801 1.62019537 1.6216863
  1.63388365 1.64781727 1.64920069 1.66755279]]


In [132]:
print(indices)

[[  1 168  15 473 572  78 561  11 682 617]]


In [23]:
df = pd.read_csv("NewData.csv")

df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,Thể thao,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,Thể thao,"['C.Ronaldo', 'Christiano Ronaldo', 'Bạn gái C..."
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,Thể thao,"['SEA Games', 'Trọng tài', 'SEA Games 31', 'SE..."
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,Thể thao,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."


In [17]:
import ast

def convert_to_list(tags_str):
    try:
        tags_list = ast.literal_eval(tags_str)
        if isinstance(tags_list, list):
            return tags_list
        else:
            return None
    except:
        return None

df['Tags'] = df['Tags'].apply(convert_to_list)

In [40]:
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,1,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,1,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,1,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),1,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,1,['An Quý Villa']
...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,8,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,8,"['C.Ronaldo', 'Christiano Ronaldo', 'Bạn gái C..."
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,8,"['SEA Games', 'Trọng tài', 'SEA Games 31', 'SE..."
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,8,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."


In [19]:
def format_tags(tags):
    tags_str = 'tags:\n'
    for tag in tags:
        tags_str += f'    - {tag}\n'
    return tags_str

df['Tags'] = df['Tags'].apply(format_tags)


In [20]:
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,Bất động sản,tags:\n - Lâm Đồng\n - Ngọc Trinh\n -...
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,Bất động sản,tags:\n - Thủ Thiêm\n - Đấu giá đất\n
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,tags:\n - Dự án\n - Nhà ở xã hội\n - ...
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),Bất động sản,tags:\n - Giấy phép xây dựng\n - Hồ sơ h...
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,Bất động sản,tags:\n - An Quý Villa\n
...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,Thể thao,tags:\n - Hoàng Anh Gia Lai\n - Kiatisak...
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,Thể thao,tags:\n - C.Ronaldo\n - Christiano Ronal...
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,Thể thao,tags:\n - SEA Games\n - Trọng tài\n -...
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,Thể thao,tags:\n - Hoàng Anh Gia Lai\n - Kiatisak...


In [21]:
df['Contents'] = df['Contents'].str.replace('  ', '\n\n', regex=False)

In [22]:
import re

df['title'] = df['Title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [159]:
!pip install unidecode
import unidecode

df['title'] = df['title'].apply(lambda x: unidecode.unidecode(x).replace('\n', ' '))
df['title'] = df['title'].apply(lambda x: unidecode.unidecode(x).replace(' ', '-'))
df['Title'] = df['Title'].str.replace('"', '', regex=False)
df['Title'] = df['Title'].str.replace("'", '', regex=False)
df['Title'] = df['Title'].str.replace("“", '', regex=False)
df['Author(s)'] = df['Author(s)'].str.replace('"', '', regex=False)
df['Author(s)'] = df['Author(s)'].str.replace("'", '', regex=False)
df['Author(s)'] = df['Author(s)'].str.replace("“", '', regex=False)
df['Summary'] = df['Summary'].str.replace('"', '', regex=False)
df['Summary'] = df['Summary'].str.replace("'", '', regex=False)
df['Summary'] = df['Summary'].str.replace("“", '', regex=False)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\votha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [160]:
df

,Title,Summary,Contents,Date,Author(s),Category,Tags,title
0,Thông tin Ngọc Trinh mua đất ở Bảo Lộc chỉ là ...,"Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,Bất động sản,tags:\n - Lâm Đồng\n - Ngọc Trinh\n -...,Thong-tin-Ngoc-Trinh-mua-dat-o-Bao-Loc-chi-la-...
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,Bất động sản,tags:\n - Thủ Thiêm\n - Đấu giá đất\n,Lo-hong-trong-viec-tham-tra-nang-luc-tai-chinh...
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,tags:\n - Dự án\n - Nhà ở xã hội\n - ...,Som-hoan-thien-cac-du-an-nha-o-xa-hoi-de-CNLD-...
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),Bất động sản,tags:\n - Giấy phép xây dựng\n - Hồ sơ h...,Chi-tiet-ho-so-hoan-cong-nha-o-nam-2022
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,Bất động sản,tags:\n - An Quý Villa\n,Khoi-tao-khong-gian-song-dang-cap-don-song-dau...
...,...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,Thể thao,tags:\n - Hoàng Anh Gia Lai\n - Kiatisak...,Hoang-Anh-Gia-Lai-cat-ngoai-binh-dau-Sydney-FC
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,Thể thao,tags:\n - C.Ronaldo\n - Christiano Ronal...,Dong-doi-Man-United-chia-se-noi-dau-mat-con-vo...
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,Thể thao,tags:\n - SEA Games\n - Trọng tài\n -...,Hon-3000-trong-tai-lam-nhiem-vu-tai-SEA-Games-31
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,Thể thao,tags:\n - Hoàng Anh Gia Lai\n - Kiatisak...,Vi-sao-gap-Sydney-FC-mang-den-nhieu-co-hoi-cho...


In [166]:
re = pd.read_csv("ketQua.csv", header=None)
re

,0,1,2,3,4,5,6,7,8,9
0,0,869,925,413,713,373,629,439,658,1018
1,1,168,15,473,572,78,561,11,682,617
2,2,231,22,535,327,17,456,630,111,1013
3,3,340,552,461,644,174,1092,306,102,961
4,4,807,806,895,1034,9,1012,1002,512,1009
...,...,...,...,...,...,...,...,...,...,...
10005,10005,9645,10009,9680,9860,9992,9949,9729,9775,9677
10006,10006,9725,9417,9415,9295,9165,9203,9253,9705,9150
10007,10007,9163,9686,9583,9982,9381,9820,9378,9955,9619
10008,10008,9997,9638,10000,9620,9818,9657,9813,9636,9908


In [167]:
for i, row in df.iloc[:-1].iterrows():
    with open(f'{row["title"]}.md', 'w', encoding='utf-8') as md_file:
        md_file.write('---\n')
        md_file.write(f'id: \'{i}\'\n')
        md_file.write(f'related_1: \'{re.iloc[i - 1, 1] + 1 }\'\n')
        md_file.write(f'related_2: \'{re.iloc[i - 1, 2] + 1 }\'\n')
        md_file.write(f'related_3: \'{re.iloc[i - 1, 3] + 1 }\'\n')
        md_file.write(f'related_4: \'{re.iloc[i - 1, 4] + 1 }\'\n')
        md_file.write(f'related_5: \'{re.iloc[i - 1, 5] + 1 }\'\n')
        md_file.write(f'related_6: \'{re.iloc[i - 1, 6] + 1 }\'\n')
        md_file.write(f'related_7: \'{re.iloc[i - 1, 7] + 1 }\'\n')
        md_file.write(f'related_8: \'{re.iloc[i - 1, 8] + 1 }\'\n')
        md_file.write(f'related_9: \'{re.iloc[i - 1, 9] + 1 }\'\n')
        md_file.write(f'postFormat: \'standard\'\n')
        md_file.write(f'title: \'{row["Title"]}\'\n')
        md_file.write(f'excerpt: \'{row["Summary"]}\'\n')
        md_file.write(f'featureImg: \'/images/posts/post_1.jpg\'\n')
        md_file.write(f'date: \'{row["Date"]}\'\n')
        md_file.write(f'cate: \'{row["Category"]}\'\n')
        md_file.write(f'cate_bg: \'bg-color-blue-one\'\n')
        md_file.write(f'cate_img: \'/images/category/technology.png\'\n')
        md_file.write(f'post_views: \'0 Views\'\n')
        md_file.write(f'post_share: \'0 Shares\'\n')
        md_file.write(f'author_name: \'{row["Author(s)"]}\'\n')
        md_file.write(f'author_desg: \'Publisher\'\n')
        md_file.write(f'author_img: \'/images/author/ashley_graham.png\'\n')
        md_file.write(f'author_bio: \'At 29 years old, my favorite compliment is being told that I look like my mom. Seeing myself in her image, like this daughter up top, makes me so proud of how far I’ve come, and so thankful for where I come from.\'\n')
        md_file.write(f'author_social:\n    -\n        icon: fab fa-facebook-f\n        url: https://facebook.com/\n    -\n        icon: fab fa-twitter\n        url: https://twitter.com/\n')
        md_file.write(f'{row["Tags"]}\n')
        md_file.write(f'---\n\n')
        md_file.write(f'{row["Contents"]}')

In [60]:
df.iloc[2652]

Title        Gợi ý đáp án môn Toán vào lớp 10 chuyên Sư phạ...
Summary      Chiều 31.5, hơn 5.300 thí sinh bắt đầu kỳ tuyể...
Contents     Các thí sinh đã hoàn thành môn Toán với thời g...
Date                         Thứ năm, 01/06/2017 10:41 (GMT+7)
Author(s)                                                  P.V
Category                                              Giáo dục
Tags         tags:\n    - Hà Nội\n    - Tuyển sinh\n    - T...
title        Goi-y-dap-an-mon-Toan-vao-lop-10-chuyen-Su-pha...
Name: 2653, dtype: object

In [61]:
df = backup.copy()


In [24]:
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,Thể thao,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,Thể thao,"['C.Ronaldo', 'Christiano Ronaldo', 'Bạn gái C..."
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,Thể thao,"['SEA Games', 'Trọng tài', 'SEA Games 31', 'SE..."
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,Thể thao,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."


In [25]:
# Tạo một từ điển ánh xạ giá trị cũ sang giá trị mới
mapping = {'Bất động sản': 1, 'Xã hội': 2, 'Kinh doanh': 3, 'Thời sự': 4, 'Pháp luật': 5, 'Giáo dục': 6, 'Sức khỏe': 7, 'Thể thao': 8, 'Công nghệ': 9, 'Du lịch': 10}

# Thay đổi giá trị của cột Category
df['Category'] = df['Category'].replace(mapping)

In [29]:
df

,Title,Summary,Contents,Date,Author(s),Category,Tags
0,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",PHƯƠNG NHIÊN,1,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",GIA MIÊU,1,"['Thủ Thiêm', 'Đấu giá đất']"
2,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,1,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",KIM NHUNG (T/H),1,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",HUYỀN NGUYỄN,1,['An Quý Villa']
...,...,...,...,...,...,...,...
10005,Hoàng Anh Gia Lai cất ngoại binh đấu Sydney FC,Huấn luyện viên Kiatisak chỉ thực hiện một sự ...,"Theo đó, huấn luyện viên Kiatisak quyết định c...","Thứ ba, 19/04/2022 17:03 (GMT+7)",THANH VŨ,8,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."
10006,Đồng đội Man United chia sẻ nỗi đau mất con vớ...,Các đồng đội ở Man United cùng nhiều ngôi sao ...,"Vào tháng 10 năm ngoái, Cristiano Ronaldo và b...","Thứ ba, 19/04/2022 17:02 (GMT+7)",CHI TRẦN,8,"['C.Ronaldo', 'Christiano Ronaldo', 'Bạn gái C..."
10007,Hơn 3.000 trọng tài làm nhiệm vụ tại SEA Games 31,Hơn 3.000 trọng tài quốc tế và quốc nội sẽ tha...,SEA Games 31 sẽ chính thức khởi tranh tại Việt...,"Thứ ba, 19/04/2022 16:59 (GMT+7)",AN NGUYÊN,8,"['SEA Games', 'Trọng tài', 'SEA Games 31', 'SE..."
10008,Vì sao gặp Sydney FC mang đến nhiều cơ hội cho...,Đội bóng của Australia có khá nhiều trụ cột lớ...,Sydney FC bước vào trận đấu gặp Hoàng Anh Gia ...,"Thứ ba, 19/04/2022 16:10 (GMT+7)",NGUYỄN ĐĂNG,8,"['Hoàng Anh Gia Lai', 'Kiatisak', 'AFC Champio..."


In [37]:
def create_insert_query(row):
    return f"INSERT INTO Api_posts (id, title, excerpt, dateCreate, category_id, views, author, content, isEnabled, idUser_id) VALUES ({row.name + 1}, '{row['Title']}', '{row['Summary']}', '{row['Date']}', '{row['Category']}', 0, '{row['Author(s)']}', '{row['Contents']}', 1, 1);"


In [38]:
new_df = df.head(100)
insert_queries = new_df.apply(create_insert_query, axis=1)

In [39]:
with open('sql.txt', 'w', encoding='utf-8') as f:
    for query in insert_queries:
        f.write(query + '\n')

In [ ]:
##END